As we uploaded all solutions and realized, bag or words with logistic regression gives us the best results, with the score of 92.83, we will use hyperparameter optimization technique on the same model and will try to check if the model fine tunes and give more better results. 

First start with data preparation. We have already discussed in the previous notebooks, how we cleaned the datasets. 

In [1]:
! pip install tldextract

     |████████████████████████████████| 87 kB 955 kB/s eta 0:00:01
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import numpy as np
from tldextract import extract
import re,sys,pickle

In [7]:
#nltk libraries
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
stop=set(stopwords.words('english'))
from nltk.stem.snowball import SnowballStemmer

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
import warnings
warnings.filterwarnings('ignore')

In [9]:
train_df = pd.read_csv('../input/zsdataset/train.csv',encoding= 'ISO-8859-1')
test_df = pd.read_csv('../input/zsdataset/test.csv',encoding= 'ISO-8859-1')

#extracts the url from the link, we will extract hostname only
def extract_url(x):
    tsd, td, tsu = extract(x) # prints abc, hostname, com
    return td

def classes_def(x):
    if x ==  "FACEBOOK":
        return "Facebook"
    elif x == 'FORUMS':
        return 'Forums'
    elif x == 'BLOG':
        return 'Blog'
    elif x == 'YOUTUBE':
        return 'Youtube'
    else:
        return 'Facebook'

def clean_text(x):
    normalizedsentense = x.lower()
    text = re.sub(r"[^a-z']+", ' ', normalizedsentense)
    return text


stemmer = SnowballStemmer('english')
def stemming(sentence):
    word_list = nltk.word_tokenize(sentence)
    stemmed_output = ' '.join([stemmer.stem(w) for w in word_list])
    return stemmed_output

train_df['Source']=train_df['Source'].apply(lambda x:classes_def(x))
test_df['Source']=test_df['Source'].apply(lambda x:classes_def(x))
train_df["Host"].fillna(train_df["Link"], inplace=True)
test_df["Host"].fillna(test_df["Link"], inplace=True)
train_df['Host']=train_df['Host'].apply(lambda x:extract_url(x))
test_df['Host']=test_df['Host'].apply(lambda x:extract_url(x))
train_df.loc[train_df.Host == '' , 'Host'] = 'youtube'

train_df = train_df.drop(['Link','time(GMT)','Title'],axis= 1)
test_df = test_df.drop(['Link','time(GMT)','Title'],axis= 1)

train_df['Date(ET)'] = pd.to_datetime(train_df['Date(ET)'],errors='coerce').dt.date
test_df['Date(ET)'] = pd.to_datetime(test_df['Date(ET)'],errors='coerce').dt.date

train_df['Time(ET)'] = pd.to_datetime(train_df['Time(ET)'],errors='coerce').dt.time
test_df['Time(ET)'] = pd.to_datetime(test_df['Time(ET)'],errors='coerce').dt.time

test_df.drop(columns = ['Index','Unnamed: 9'],inplace = True)

train_df = train_df[train_df['TRANS_CONV_TEXT'].notna()]

train_df['TRANS_CONV_TEXT']=train_df['TRANS_CONV_TEXT'].apply(lambda x:clean_text(x))
test_df['TRANS_CONV_TEXT']=test_df['TRANS_CONV_TEXT'].apply(lambda x:clean_text(x))

train_df['TRANS_CONV_TEXT']=train_df['TRANS_CONV_TEXT'].apply(lambda x: stemming(x))
test_df['TRANS_CONV_TEXT']=test_df['TRANS_CONV_TEXT'].apply(lambda x: stemming(x))

In [10]:
train_df.head()

,Source,Host,Date(ET),Time(ET),TRANS_CONV_TEXT,Patient_Tag
0,Forums,cafepharma,2016-06-15,13:58:00,i do n't disagre with you in principl i 'm jus...,0
1,Forums,patient,2016-05-07,NaT,i am alway dizzi i get dizzi stand up so i hav...,1
2,Blog,abcnewsradioonline,2016-04-14,15:00:38,axell bauer griffin filmmag new york queen lat...,0
3,Forums,cancer-forums,2016-06-18,20:46:00,i am and i have been throw up for about a year...,1
4,Forums,diyaudio,2016-06-15,03:26:00,quot origin post by boyan silyavski wake up my...,0


In [11]:
test_df.head()

,Source,Host,Date(ET),Time(ET),TRANS_CONV_TEXT
0,Blog,uhmagonline,2016-07-30,00:41:23,babi slice the son of the late kimbo slice has...
1,Forums,yuku,2016-06-20,00:07:30,p font face san serif size i have had both sil...
2,Blog,blogspot,2016-06-15,15:44:00,previous sodium glucos cotransport sglt inhibi...
3,Forums,healthboards,2016-07-17,19:41:00,hello i suffer from congest heart failur due t...
4,Blog,sciencecodex,2016-04-04,15:30:45,a daili dose of vitamin d improv heart functio...


In Logistic Regression there are 2 papameters we can consider for hyperparamter tuning. 
* penalty{‘l1’, ‘l2’, ‘elasticnet’, ‘none’}, default=’l2’ Used to specify the norm used in the penalization. The ‘newton-cg’, ‘sag’ and ‘lbfgs’ solvers support only l2 penalties. ‘elasticnet’ is only supported by the ‘saga’ solver. If ‘none’ (not supported by the liblinear solver), no regularization is applied.
* Cfloat, default=1.0, Inverse of regularization strength; must be a positive float. Like in support vector machines, smaller values specify stronger regularization.

* C values [0.001, 0.01, 0.1, 1 , 10 ,100]
* Also, look in the above point 1, when there is l2 penelty we can apply solvers ['newton-cg', 'lbfgs', 'liblinear']

In [19]:
df_hpo = train_df[['TRANS_CONV_TEXT','Patient_Tag']]
df_hpo.head()

,TRANS_CONV_TEXT,Patient_Tag
0,i do n't disagre with you in principl i 'm jus...,0
1,i am alway dizzi i get dizzi stand up so i hav...,1
2,axell bauer griffin filmmag new york queen lat...,0
3,i am and i have been throw up for about a year...,1
4,quot origin post by boyan silyavski wake up my...,0


In [39]:
from sklearn import linear_model
from sklearn import metrics
from sklearn import model_selection
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import make_scorer

if __name__ =="__main__":
    
    
    #randomize the data
    df_hpo = df_hpo.sample(frac = 1).reset_index(drop =True)
    
    #initialize the count vector tokenizer with NLTK word tokenize
    count_vec = CountVectorizer(tokenizer = word_tokenize, token_pattern = None)
    
    #fit the count vector
    count_vec.fit(df_hpo.TRANS_CONV_TEXT)
    pickle.dump(count_vec, open("count_vectorizer.pickle", "wb")) 
    
    #transform the training and validation data reviews
    X = count_vec.transform(df_hpo.TRANS_CONV_TEXT)
    y = df_hpo.Patient_Tag.values
    
    classifier = linear_model.LogisticRegression()
    param_grid = {
                    "solver": ['newton-cg','lbfgs','sag','saga'],
                    "penalty": ['l2'],
                    "C": [100, 10, 1.0, 0.1, 0.01, 0.001]
                }
    roc_auc_weighted = metrics.make_scorer(metrics.roc_auc_score, average='weighted')
    model = model_selection.GridSearchCV(
                        estimator=classifier,
                        param_grid=param_grid,
                        scoring=roc_auc_weighted,
                        verbose=10,
                        n_jobs=1,
                        cv=5
                        ) 
  
    model.fit(X, y)
    print(f"Best score: {model.best_score_}")
    print("Best parameters set:")
    best_parameters = model.best_estimator_.get_params()
    for param_name in sorted(param_grid.keys()):
        print(f"\t{param_name}: {best_parameters[param_name]}")
    


Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV] C=100, penalty=l2, solver=newton-cg .............................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] . C=100, penalty=l2, solver=newton-cg, score=0.840, total=   0.2s
[CV] C=100, penalty=l2, solver=newton-cg .............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV] . C=100, penalty=l2, solver=newton-cg, score=0.838, total=   0.3s
[CV] C=100, penalty=l2, solver=newton-cg .............................


...

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    2.9s remaining:    0.0s


[CV] ..... C=100, penalty=l2, solver=lbfgs, score=0.845, total=   0.4s
[CV] C=100, penalty=l2, solver=sag ...................................
[CV] ....... C=100, penalty=l2, solver=sag, score=0.858, total=   0.2s
[CV] C=100, penalty=l2, solver=sag ...................................
[CV] ....... C=100, penalty=l2, solver=sag, score=0.854, total=   0.1s
[CV] C=100, penalty=l2, solver=sag ...................................
[CV] ....... C=100, penalty=l2, solver=sag, score=0.841, total=   0.1s
[CV] C=100, penalty=l2, solver=sag ...................................
[CV] ....... C=100, penalty=l2, solver=sag, score=0.868, total=   0.1s
[CV] C=100, penalty=l2, solver=sag ...................................
[CV] ....... C=100, penalty=l2, solver=sag, score=0.780, total=   0.1s
[CV] C=100, penalty=l2, solver=saga ..................................
[CV] ...... C=100, penalty=l2, solver=saga, score=0.840, total=   0.2s
[CV] C=100, penalty=l2, solver=saga ..................................
[CV] .

[CV] ..... C=0.1, penalty=l2, solver=lbfgs, score=0.853, total=   0.3s
[CV] C=0.1, penalty=l2, solver=lbfgs .................................
[CV] ..... C=0.1, penalty=l2, solver=lbfgs, score=0.835, total=   0.3s
[CV] C=0.1, penalty=l2, solver=lbfgs .................................
[CV] ..... C=0.1, penalty=l2, solver=lbfgs, score=0.820, total=   0.3s
[CV] C=0.1, penalty=l2, solver=sag ...................................
[CV] ....... C=0.1, penalty=l2, solver=sag, score=0.858, total=   0.2s
[CV] C=0.1, penalty=l2, solver=sag ...................................
[CV] ....... C=0.1, penalty=l2, solver=sag, score=0.854, total=   0.2s
[CV] C=0.1, penalty=l2, solver=sag ...................................
[CV] ....... C=0.1, penalty=l2, solver=sag, score=0.841, total=   0.1s
[CV] C=0.1, penalty=l2, solver=sag ...................................
[CV] ....... C=0.1, penalty=l2, solver=sag, score=0.868, total=   0.1s
[CV] C=0.1, penalty=l2, solver=sag ...................................
[CV] .

[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:   24.8s finished


Best score: 0.8600706818721786
Best parameters set:
	C: 1.0
	penalty: l2
	solver: newton-cg


Now let's take penalty l1 and solvers ‘liblinear’ and ‘saga’ 
as they handle l1 penalty. 
and check if they improves the results

In [43]:
from sklearn import linear_model
from sklearn import metrics
from sklearn import model_selection
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import make_scorer

if __name__ =="__main__":
    
    
    #randomize the data
    df_hpo = df_hpo.sample(frac = 1).reset_index(drop =True)
    
    #initialize the count vector tokenizer with NLTK word tokenize
    count_vec = CountVectorizer(tokenizer = word_tokenize, token_pattern = None)
    
    #fit the count vector
    count_vec.fit(df_hpo.TRANS_CONV_TEXT)
    pickle.dump(count_vec, open("count_vectorizer.pickle", "wb")) 
    
    #transform the training and validation data reviews
    X = count_vec.transform(df_hpo.TRANS_CONV_TEXT)
    y = df_hpo.Patient_Tag.values
    
    classifier = linear_model.LogisticRegression()
    param_grid = {
                    "solver": ['saga','liblinear'],
                    "penalty": ['l1'],
                    "C": [100, 10, 1.0, 0.1, 0.01, 0.001]
                }
    roc_auc_weighted = metrics.make_scorer(metrics.roc_auc_score, average='weighted')
    model = model_selection.GridSearchCV(
                        estimator=classifier,
                        param_grid=param_grid,
                        scoring=roc_auc_weighted,
                        verbose=10,
                        n_jobs=1,
                        cv=5
                        ) 
  
    model.fit(X, y)
    print(f"Best score: {model.best_score_}")
    print("Best parameters set:")
    best_parameters = model.best_estimator_.get_params()
    for param_name in sorted(param_grid.keys()):
        print(f"\t{param_name}: {best_parameters[param_name]}")
    


Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] C=100, penalty=l1, solver=saga ..................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ...... C=100, penalty=l1, solver=saga, score=0.718, total=   6.0s
[CV] C=100, penalty=l1, solver=saga ..................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.0s remaining:    0.0s


[CV] ...... C=100, penalty=l1, solver=saga, score=0.869, total=   6.2s
[CV] C=100, penalty=l1, solver=saga ..................................


...

[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:   25.0s remaining:    0.0s


[CV] ...... C=100, penalty=l1, solver=saga, score=0.847, total=   6.5s
[CV] C=100, penalty=l1, solver=liblinear .............................
[CV] . C=100, penalty=l1, solver=liblinear, score=0.871, total=   0.1s
[CV] C=100, penalty=l1, solver=liblinear .............................
[CV] . C=100, penalty=l1, solver=liblinear, score=0.879, total=   0.1s
[CV] C=100, penalty=l1, solver=liblinear .............................
[CV] . C=100, penalty=l1, solver=liblinear, score=0.882, total=   0.0s
[CV] C=100, penalty=l1, solver=liblinear .............................
[CV] . C=100, penalty=l1, solver=liblinear, score=0.804, total=   0.0s
[CV] C=100, penalty=l1, solver=liblinear .............................


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   31.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:   31.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:   31.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:   31.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   31.7s remaining:    0.0s


[CV] . C=100, penalty=l1, solver=liblinear, score=0.897, total=   0.0s
[CV] C=10, penalty=l1, solver=saga ...................................
[CV] ....... C=10, penalty=l1, solver=saga, score=0.718, total=   4.5s
[CV] C=10, penalty=l1, solver=saga ...................................
[CV] ....... C=10, penalty=l1, solver=saga, score=0.879, total=   4.7s
[CV] C=10, penalty=l1, solver=saga ...................................
[CV] ....... C=10, penalty=l1, solver=saga, score=0.825, total=   4.7s
[CV] C=10, penalty=l1, solver=saga ...................................
[CV] ....... C=10, penalty=l1, solver=saga, score=0.885, total=   5.0s
[CV] C=10, penalty=l1, solver=saga ...................................
[CV] ....... C=10, penalty=l1, solver=saga, score=0.847, total=   4.8s
[CV] C=10, penalty=l1, solver=liblinear ..............................
[CV] .. C=10, penalty=l1, solver=liblinear, score=0.830, total=   0.1s
[CV] C=10, penalty=l1, solver=liblinear ..............................
[CV] .

[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:  1.1min finished


With l1 penalty, c= 100 and liblinear solver we get 86.65 rocauc our best score. Which is 0.65 higher than l1 penalty. So we will impliment the best parameters and check the results.

In [40]:
df = train_df.copy()
df_test = test_df.copy()

from sklearn import linear_model
from sklearn import metrics
from sklearn import model_selection
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score


if __name__ =="__main__":
    #create a new column with and fill it with -1
    df['kfold'] = -1
    #randomize the data
    df = df.sample(frac = 1).reset_index(drop =True)
    y = df.Patient_Tag.values
    #initiate model selection class from stratified kfold module
    kf = model_selection.StratifiedKFold(n_splits = 5)
    
    # fill the new k fold column
    for f,(t_,v_) in enumerate(kf.split(X =df,y = y)):
        df.loc[v_,'kfold'] = f

    #go over the folds created
    for fold_ in range(5):
        
        #creating temporary dataframes for train and test
        train_data = df[df.kfold != fold_].reset_index(drop = True)
        test_data = df[df.kfold == fold_].reset_index(drop = True)

        #initialize the count vector tokenizer with NLTK word tokenize
        count_vec = CountVectorizer(tokenizer = word_tokenize, token_pattern = None)
        
        #fit the count vector
        count_vec.fit(train_data.TRANS_CONV_TEXT)
        pickle.dump(count_vec, open("count_vectorizer.pickle", "wb")) 
        
        #transform the training and validation data reviews
        xtrain = count_vec.transform(train_data.TRANS_CONV_TEXT)
        xtest = count_vec.transform(test_data.TRANS_CONV_TEXT)


        model = linear_model.LogisticRegression(C = 100, penalty = 'l1',solver = 'liblinear')
        model.fit(xtrain,train_data.Patient_Tag)
        
        preds = model.predict(xtest)
        roc_auc_score = metrics.roc_auc_score(test_data.Patient_Tag,preds)
        
        print(f'fold : {fold_}')
        print(f'roc_auc = {roc_auc_score}')
        print(classification_report(test_data.Patient_Tag,preds))
        print(f"Accuracy Score = {accuracy_score(test_data.Patient_Tag,preds)}")
              
        print('<<<<<<<<<<<<<<<<<<<<<<>>>>>>>>>>>>>>>>>>>')


fold : 0
roc_auc = 0.8301630434782609
              precision    recall  f1-score   support

           0       0.92      0.97      0.95       184
           1       0.87      0.69      0.77        48

    accuracy                           0.91       232
   macro avg       0.90      0.83      0.86       232
weighted avg       0.91      0.91      0.91       232

Accuracy Score = 0.9137931034482759
<<<<<<<<<<<<<<<<<<<<<<>>>>>>>>>>>>>>>>>>>
fold : 1
roc_auc = 0.8350409836065574
              precision    recall  f1-score   support

           0       0.93      0.96      0.94       183
           1       0.83      0.71      0.76        48

    accuracy                           0.91       231
   macro avg       0.88      0.84      0.85       231
weighted avg       0.91      0.91      0.91       231

Accuracy Score = 0.9090909090909091
<<<<<<<<<<<<<<<<<<<<<<>>>>>>>>>>>>>>>>>>>
fold : 2
roc_auc = 0.8536543715846994
              precision    recall  f1-score   support

           0       0.

It seems the default parameters are the best for our model. 
We will take the submission file. 

In [41]:
df_test = test_df.copy()

In [42]:
count_vec = pickle.load(open("./count_vectorizer.pickle", 'rb'))
test_features = count_vec.transform(df_test.TRANS_CONV_TEXT)
test_preds = model.predict(test_features)
submission_logistic_hpo = pd.read_csv('../input/zsdataset/test.csv',encoding = 'ISO-8859-1')
submission = pd.DataFrame()
submission['Index'] = submission_logistic_hpo['Index']
submission['Patient_Tag'] = test_preds
submission.to_csv('submission_logistic_bow_hpo.csv',index = False)

# References 
* sklearn website 
* https://www.amazon.de/Approaching-Almost-Machine-Learning-Problem/dp/8269211508/ref=asc_df_8269211508/?tag=googshopde21&linkCode=df0&hvadid=447482931749&hvpos=&hvnetw=g&hvrand=1474972248643818998&hvpone=&hvptwo=&hvqmt=&hvdev=c&hvdvcmdl=&hvlocint=&hvlocphy=9042923&hvtargid=pla-922436928846&psc=1&th=1&psc=1